In [2]:
import requests
import json

HOST = "http://10.38.2.35:8000/"
API_CANCLED = "canclled"
API_PNR_RANKING = "pnr_ranking?flight_id="
API_ALT_FLIGHT="alt_flight?flight_id="

def get_cancled():
    response = requests.request("GET", HOST + API_CANCLED, headers={}, data={})
    return json.loads(response.text)

def get_pnrs(flight_id):
    response = requests.request("GET", HOST + API_PNR_RANKING + str(flight_id), headers={}, data={})
    return json.loads(response.text)

def get_alt_flights(flight_id):
    response = requests.request("GET", HOST + API_ALT_FLIGHT + str(flight_id), headers={}, data={})
    return json.loads(response.text)

In [3]:
cancel=get_cancled()

In [204]:
cancel['data'][0]
map_cabin={ 'F':'F','P':'F','C':'B','J':'B','Z':'B','Q':'P','R':'P','S':'P','T':'P','H':'P','M':'P',
          'Y':'E', 'A':'E', 'B':'E', 'D':'E', 'E':'E', 'G':'E', 'I':'E', 'K':'E', 'L':'E', 'N':'E', 'O':'E',
           'U':'E', 'V':'E','W':'E','X':'E'}

In [205]:
# get_alt_flights('c53865e9')

In [206]:
pnr={}
alt_flight={}
for f in cancel['data']:
    pnr[f['flight_id']]=get_pnrs(f['flight_id'])['data']
    alt=get_alt_flights(f['flight_id'])['data']
    f2=[]
    for flight in alt:
        f3={}
        f3['flight_id']=flight['flight_id']
        f3['F']=0
        f3['B']=0
        f3['P']=0
        f3['E']=0
#         print()
        for key in map_cabin:
            if key in flight['avilable_seats']:
                f3[map_cabin[key]]+=flight['avilable_seats'][key]
        f2.append(f3)
    alt_flight[f['flight_id']]=f2

In [207]:
for i in alt_flight:
    print(alt_flight[i])
    break
    

[{'flight_id': 'b394dfd1', 'F': 9, 'B': 13, 'P': 13, 'E': 44}, {'flight_id': 'f8245c5d', 'F': 9, 'B': 10, 'P': 16, 'E': 40}, {'flight_id': '968a3b1c', 'F': 7, 'B': 11, 'P': 20, 'E': 32}, {'flight_id': 'b613bdb5', 'F': 0, 'B': 0, 'P': 0, 'E': 2}, {'flight_id': 'b80f7b86', 'F': 0, 'B': 1, 'P': 1, 'E': 2}, {'flight_id': 'b3d957da', 'F': 7, 'B': 16, 'P': 16, 'E': 51}, {'flight_id': '80d06f5f', 'F': 2, 'B': 0, 'P': 2, 'E': 1}, {'flight_id': '973975d2', 'F': 1, 'B': 0, 'P': 1, 'E': 3}, {'flight_id': '09b66d3f', 'F': 1, 'B': 0, 'P': 2, 'E': 2}, {'flight_id': '2eefb056', 'F': 2, 'B': 0, 'P': 4, 'E': 4}, {'flight_id': '21d90317', 'F': 1, 'B': 0, 'P': 3, 'E': 2}, {'flight_id': '8fa60bff', 'F': 2, 'B': 4, 'P': 3, 'E': 3}, {'flight_id': 'e7df70dd', 'F': 0, 'B': 2, 'P': 0, 'E': 1}, {'flight_id': 'a742b2f3', 'F': 0, 'B': 1, 'P': 1, 'E': 4}, {'flight_id': 'bce685b5', 'F': 9, 'B': 0, 'P': 10, 'E': 23}, {'flight_id': '5ef641be', 'F': 7, 'B': 0, 'P': 7, 'E': 22}, {'flight_id': 'bb8e6831', 'F': 1, 'B': 2

In [256]:
used_seat={}

for i in alt_flight:
    for j in alt_flight[i]:
        used_seat[j['flight_id']]={'F':0,'B':0,'P':0,'E':0}
#             print(j)
        
used_seat

{'b394dfd1': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'f8245c5d': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '968a3b1c': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'b613bdb5': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'b80f7b86': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'b3d957da': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '80d06f5f': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '973975d2': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '09b66d3f': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '2eefb056': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '21d90317': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '8fa60bff': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'e7df70dd': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'a742b2f3': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'bce685b5': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '5ef641be': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'bb8e6831': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '1817d1a9': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '826fce02': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '49275e19': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'f02572c1': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '9cc3c76c': 

In [ ]:
pnr

In [ ]:
needed_seat=[]
for i in pnr:
    alloc={}
    for j in pnr[i]:
        alloc=j;
        alloc['flight_id']=i
        needed_seat.append(alloc)

In [246]:
sorted_pnr = sorted(needed_seat, key=lambda x: x['score'],reverse=True)

In [247]:
allocated={}

for i in sorted_pnr:
    allocated[i['pnr']]='NULL'
    

In [248]:
index_pnr=0
flight_index={i : {'F':0,'B':0,'P':0,'E':0} for i in pnr}
# flight_index
used_seat

{'b394dfd1': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'f8245c5d': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '968a3b1c': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'b613bdb5': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'b80f7b86': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'b3d957da': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '80d06f5f': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '973975d2': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '09b66d3f': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '2eefb056': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '21d90317': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '8fa60bff': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'e7df70dd': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'a742b2f3': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'bce685b5': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '5ef641be': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'bb8e6831': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '1817d1a9': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '826fce02': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '49275e19': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'f02572c1': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '9cc3c76c': 

In [249]:
allocated

{'8e4a17': 'NULL',
 'f87a3d': 'NULL',
 '6f8d9c': 'NULL',
 'c65e4d': 'NULL',
 '5fc3e8': 'NULL',
 '895ee8': 'NULL',
 '105e86': 'NULL',
 '83b0d7': 'NULL',
 '7b9876': 'NULL',
 'e6a0fd': 'NULL',
 'c40f05': 'NULL',
 'ce950f': 'NULL',
 '38ab49': 'NULL',
 '2383b0': 'NULL',
 'fccab6': 'NULL',
 'd127d8': 'NULL',
 'c3e5e0': 'NULL',
 '50e663': 'NULL',
 '4a387b': 'NULL',
 '58e903': 'NULL',
 'd82291': 'NULL',
 '49939c': 'NULL',
 'cb6705': 'NULL',
 '0d26a3': 'NULL',
 '75a90a': 'NULL',
 '59a1cf': 'NULL',
 '188cbb': 'NULL',
 '22aad0': 'NULL',
 'b00f4a': 'NULL',
 '3550c0': 'NULL',
 'bdfd1c': 'NULL',
 '7beac3': 'NULL',
 'b0bf96': 'NULL',
 '9a4117': 'NULL',
 '8b1296': 'NULL',
 '942dbc': 'NULL',
 '2dae7e': 'NULL',
 'c1374e': 'NULL',
 '2ab702': 'NULL',
 '262470': 'NULL',
 'ead910': 'NULL',
 '8a5234': 'NULL',
 '747db0': 'NULL',
 '933c10': 'NULL',
 '292f5d': 'NULL',
 'f4a910': 'NULL',
 'a558e8': 'NULL',
 'f8c0cf': 'NULL',
 '8ab748': 'NULL',
 '32aae0': 'NULL',
 '8dec0e': 'NULL',
 '88e7e1': 'NULL',
 'aa3587': '

In [250]:
# ik=0
flights=4
cabins=['F','B','P','E']
cabin_ind={'F':0 , 'B':1,'P':2,'E':3}
for i in sorted_pnr:
#     ik+=1
    
#     if(ik>5):
#         break
#     print(i)
    f=0;
    list=[]
    ok=1;
    cabin_no=cabin_ind[map_cabin[i['class']]]
    while(ok==1 and cabin_no<8):
        cabin=cabins[cabin_no%4]
        ind = 0
#     print(used_seat)
        while(ind<len(alt_flight[i['flight_id']])):
        
            max_seat=alt_flight[i['flight_id']][ind][cabin]
            alt = alt_flight[i['flight_id']][ind]['flight_id']
            print(max_seat)
            print(ind)
            print(alt)
#         print(used_seat)
        
            if(used_seat[alt][cabin]+i['pax']<=max_seat):
#                 used_seat[alt][cabin]+=i['pax'];
#                 allocated[i['pnr']]=[alt,cabins[cabin_no%4],1]
#                 print("allovc")
#                 ok=0
#                 break
                
                if(f<flights):
                    if f==0:
                        used_seat[alt][cabin]+=i['pax']
                    list.append([alt,cabins[cabin_no%4]])
                    f+=1;
                else:
                    ok=0
                    break;
        
            ind+=1
            
        cabin_no+=1
    allocated[i['pnr']]=list

0
0
248d086d
9
1
6a59eb56
allovc
7
0
d3f9b516
allovc
10
0
b5833208
allovc
1
0
2e05935e
5
1
4bf78735
allovc
1
0
a2b8853d
17
1
9986ecf8
allovc
2
0
f390a612
allovc
0
0
8ca4312b
1
1
91f857ec
0
2
38521767
1
3
75c0d838
1
4
73b893fd
0
5
e5675a83
0
6
df800d2c
7
7
e5741499
allovc
0
0
5881440c
6
0
5881440c
allovc
8
0
ec393857
allovc
0
0
7265a4e8
5
1
c3b59e2e
allovc
2
0
cbc16557
1
1
b1109988
6
2
66c5203b
allovc
10
0
51cdd9c7
allovc
1
0
005f4964
7
1
6be59b57
allovc
0
0
dfbeffed
0
1
2b2a4146
0
2
3bb7760d
0
3
152498bf
0
4
70eb89ae
0
5
1f6c579e
0
0
dfbeffed
0
1
2b2a4146
0
2
3bb7760d
0
3
152498bf
0
4
70eb89ae
0
5
1f6c579e
16
0
dfbeffed
allovc
3
0
b5833208
5
1
fbbd7eca
allovc
0
0
7265a4e8
5
1
c3b59e2e
4
2
5546d56a
allovc
0
0
a2b8853d
8
1
9986ecf8
allovc
0
0
a2b8853d
8
1
9986ecf8
allovc
0
0
a2b8853d
8
1
9986ecf8
10
2
79d5b355
allovc
10
0
e0853f8d
allovc
10
0
e0853f8d
allovc
0
0
491d2c9a
0
1
14f46d57
8
2
eadc3e73
allovc
1
0
2e05935e
5
1
4bf78735
3
2
d4c776e5
0
3
d30695cc
5
4
f3b27cfe
allovc
0
0
566d4857


dfbeffed
15
1
2b2a4146
33
2
3bb7760d
29
3
152498bf
allovc
31
0
dfbeffed
15
1
2b2a4146
33
2
3bb7760d
29
3
152498bf
allovc
31
0
dfbeffed
15
1
2b2a4146
33
2
3bb7760d
29
3
152498bf
allovc
25
0
09746a12
allovc
25
0
09746a12
allovc
25
0
09746a12
allovc
29
0
09746a12
allovc
29
0
09746a12
allovc
9
0
b5833208
15
1
fbbd7eca
9
2
be767c21
allovc
9
0
b5833208
15
1
fbbd7eca
9
2
be767c21
9
3
88b478c3
allovc
9
0
b5833208
15
1
fbbd7eca
9
2
be767c21
9
3
88b478c3
allovc
17
0
b5833208
allovc
31
0
cc407d7f
allovc
31
0
cc407d7f
allovc
31
0
cc407d7f
allovc
11
0
ae7a9f88
allovc
29
0
ae7a9f88
allovc
13
0
7265a4e8
9
1
c3b59e2e
14
2
5546d56a
8
3
0b037d0c
9
4
d8ab43c6
6
5
1d7c6c16
12
6
ef0167ff
allovc
13
0
7265a4e8
9
1
c3b59e2e
14
2
5546d56a
8
3
0b037d0c
9
4
d8ab43c6
6
5
1d7c6c16
12
6
ef0167ff
allovc
13
0
7265a4e8
9
1
c3b59e2e
14
2
5546d56a
8
3
0b037d0c
9
4
d8ab43c6
6
5
1d7c6c16
12
6
ef0167ff
18
7
6d9acb06
allovc
33
0
7265a4e8
allovc
33
0
7265a4e8
25
1
c3b59e2e
allovc
33
0
7265a4e8
25
1
c3b59e2e
allovc
33
0
7265a

5
7df6bbbc
9
6
aabc3fb1
42
7
722af1f4
5
8
5eb30a91
37
9
e0ddbe52
39
10
bdc3b0d8
allovc
5
0
a2b8853d
25
1
9986ecf8
37
2
79d5b355
13
3
d4b724ff
4
4
7cf25481
5
5
7df6bbbc
9
6
aabc3fb1
42
7
722af1f4
5
8
5eb30a91
37
9
e0ddbe52
39
10
bdc3b0d8
allovc
31
0
dfbeffed
15
1
2b2a4146
33
2
3bb7760d
29
3
152498bf
35
4
70eb89ae
32
5
1f6c579e
0
0
dfbeffed
0
1
2b2a4146
0
2
3bb7760d
0
3
152498bf
0
4
70eb89ae
0
5
1f6c579e
0
0
dfbeffed
0
1
2b2a4146
0
2
3bb7760d
0
3
152498bf
0
4
70eb89ae
0
5
1f6c579e
16
0
dfbeffed
9
1
2b2a4146
18
2
3bb7760d
12
3
152498bf
12
4
70eb89ae
16
5
1f6c579e
31
0
dfbeffed
15
1
2b2a4146
33
2
3bb7760d
29
3
152498bf
35
4
70eb89ae
32
5
1f6c579e
31
0
dfbeffed
15
1
2b2a4146
33
2
3bb7760d
29
3
152498bf
35
4
70eb89ae
32
5
1f6c579e
0
0
dfbeffed
0
1
2b2a4146
0
2
3bb7760d
0
3
152498bf
0
4
70eb89ae
0
5
1f6c579e
0
0
dfbeffed
0
1
2b2a4146
0
2
3bb7760d
0
3
152498bf
0
4
70eb89ae
0
5
1f6c579e
16
0
dfbeffed
9
1
2b2a4146
18
2
3bb7760d
12
3
152498bf
12
4
70eb89ae
16
5
1f6c579e
31
0
dfbeffed
15
1
2b2a414

5
7
56e9da4d
5
8
8dd04a69
1
9
97c38791
1
10
68fc099f
2
11
ee2e2ccf
1
12
f0f952b2
0
13
cc96fa65
3
14
02a078af
1
15
e644db2e
2
16
2d190194
7
0
d3f9b516
6
1
05075981
5
2
d00e898d
2
3
e802d965
7
4
be7dd45b
6
5
9f6c0028
2
6
188c0f00
1
7
56e9da4d
1
8
8dd04a69
4
9
97c38791
1
10
68fc099f
1
11
ee2e2ccf
4
12
f0f952b2
2
13
cc96fa65
2
14
02a078af
2
15
e644db2e
2
16
2d190194
12
0
d3f9b516
9
1
05075981
12
2
d00e898d
7
3
e802d965
11
4
be7dd45b
12
5
9f6c0028
4
6
188c0f00
3
7
56e9da4d
3
8
8dd04a69
2
9
97c38791
4
10
68fc099f
6
11
ee2e2ccf
1
12
f0f952b2
3
13
cc96fa65
3
14
02a078af
2
15
e644db2e
3
16
2d190194
12
0
d3f9b516
9
1
05075981
12
2
d00e898d
7
3
e802d965
11
4
be7dd45b
12
5
9f6c0028
4
6
188c0f00
3
7
56e9da4d
3
8
8dd04a69
2
9
97c38791
4
10
68fc099f
6
11
ee2e2ccf
1
12
f0f952b2
3
13
cc96fa65
3
14
02a078af
2
15
e644db2e
3
16
2d190194
4
0
d3f9b516
1
1
05075981
3
2
d00e898d
1
3
e802d965
1
4
be7dd45b
1
5
9f6c0028
0
6
188c0f00
2
7
56e9da4d
2
8
8dd04a69
0
9
97c38791
0
10
68fc099f
2
11
ee2e2ccf
0
12
f0f952b2

In [251]:
used_seat

{'b394dfd1': {'F': 4, 'B': 8, 'P': 5, 'E': 30},
 'f8245c5d': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '968a3b1c': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'b613bdb5': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'b80f7b86': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'b3d957da': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '80d06f5f': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '973975d2': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '09b66d3f': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '2eefb056': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '21d90317': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '8fa60bff': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'e7df70dd': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'a742b2f3': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'bce685b5': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '5ef641be': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'bb8e6831': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '1817d1a9': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '826fce02': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '49275e19': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'f02572c1': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '9cc3c76c':

In [252]:
allocated

{'8e4a17': ['6a59eb56', 'F', 1],
 'f87a3d': ['d3f9b516', 'B', 1],
 '6f8d9c': ['b5833208', 'B', 1],
 'c65e4d': ['4bf78735', 'F', 1],
 '5fc3e8': ['9986ecf8', 'P', 1],
 '895ee8': ['f390a612', 'F', 1],
 '105e86': ['e5741499', 'F', 1],
 '83b0d7': ['5881440c', 'B', 1],
 '7b9876': ['ec393857', 'F', 1],
 'e6a0fd': ['c3b59e2e', 'F', 1],
 'c40f05': ['66c5203b', 'F', 1],
 'ce950f': ['51cdd9c7', 'F', 1],
 '38ab49': ['6be59b57', 'F', 1],
 '2383b0': ['dfbeffed', 'P', 1],
 'fccab6': ['fbbd7eca', 'F', 1],
 'd127d8': ['5546d56a', 'F', 1],
 'c3e5e0': 'NULL',
 '50e663': ['9986ecf8', 'F', 1],
 '4a387b': ['9986ecf8', 'F', 1],
 '58e903': ['79d5b355', 'F', 1],
 'd82291': ['e0853f8d', 'F', 1],
 '49939c': ['e0853f8d', 'F', 1],
 'cb6705': 'NULL',
 '0d26a3': ['eadc3e73', 'F', 1],
 '75a90a': ['f3b27cfe', 'F', 1],
 '59a1cf': ['38bdc7c9', 'F', 1],
 '188cbb': ['51cdd9c7', 'F', 1],
 '22aad0': ['a81f1fff', 'F', 1],
 'b00f4a': ['a81f1fff', 'E', 1],
 '3550c0': ['28cf5c8d', 'F', 1],
 'bdfd1c': ['5cc80912', 'F', 1],
 '7be

In [253]:
sum=0
for i in allocated: 
    if allocated[i]=='NULL':
        sum+=1

In [254]:
sum


591

In [255]:
len(allocated)

4014

In [221]:
sorted

<function sorted(iterable, /, *, key=None, reverse=False)>